In [1]:
import pandas as pd
import pickle

import ipywidgets as widgets
from ipywidgets import interact

from src.data_load import load_tables, load_online_instance, load_distances
from src.filtering import flexible_filter
from src.plotting import plot_metrics_comparison, plot_gantt_labors_by_driver, plot_service_driver_distance, \
                         plot_gantt_by_services, plot_gantt_by_drivers
from src.metrics import collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df, \
                        collect_hist_baseline_dfs
from src.experimentation_config import *
from src.config import *
from src.alpha_tuning_utils import collect_alpha_results_to_df, filter_df_on_hyperparameter_selection, include_all_city, \
    metrics, iterations_nums, alphas

data_path = '../data'

instance = 'instAD1'        # Options: ['instAS1', 'instRS1']

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
distance_method = 'haversine'      # Options: ['precalced', 'haversine']

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
dist_dict = {}

fechas = fechas_dict[instance]

# Cargar resultados

In [4]:
labors_algo_offline_df, moves_algo_offline_df = collect_alpha_results_to_df(data_path, instance, distance_method, metrics, alphas, iterations_nums)

In [ ]:
hist_inst = f'{instance[:5]}S{instance[6:]}'
labors_hist_df, moves_hist_df = collect_hist_baseline_dfs(data_path, hist_inst, fechas, distance_method)

hyperparameter_selection = {
    'driver_distance': {'num_iter':500, 'alpha':0},
    'driver_extra_time':    {'num_iter':500, 'alpha':0.1},
    'hybrid':               {'num_iter':500, 'alpha':0.1}
}

labors_algo_offline_df, moves_algo_offline_df = collect_alpha_results_to_df(data_path, hist_inst, distance_method, metrics, alphas, iterations_nums)
labors_algo_offline_df = filter_df_on_hyperparameter_selection(labors_algo_offline_df, hyperparameter_selection)
moves_algo_offline_df = filter_df_on_hyperparameter_selection(moves_algo_offline_df, hyperparameter_selection)

labors_hist_df = include_all_city(labors_hist_df)
moves_hist_df = include_all_city(moves_hist_df)

with open(f'{data_path}/resultados/online_operation/{instance}/res_dynamic.pkl', "rb") as f:
    labors_algo_online_df, moves_algo_online_df = pickle.load(f)  # dict: {city: (df_cleaned, df_moves, n_drivers)}


labors_algo_online_df = include_all_city(labors_algo_online_df)
moves_algo_online_df = include_all_city(moves_algo_online_df)

# Results

In [ ]:
def plot_metrics_comparison_online_offline(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_offline_df: pd.DataFrame,
    moves_algo_offline_df: pd.DataFrame,
    labors_algo_online_df: pd.DataFrame,
    moves_algo_online_df: pd.DataFrame,
    city: str,
    metricas,
    dist_dict: dict,
    fechas: tuple[str, str], 
    group_by=None,
    xaxis_mode: str = "date"
):
    """
    Genera gráficos comparativos de métricas entre solución real y algoritmo.
    Muestra de a dos métricas por figura (side-by-side), con leyenda debajo.

    Parámetros:
    - xaxis_mode: "date" para mostrar fechas reales,
                  "day" para mostrar etiquetas tipo 'day1', 'day2', ...
    """
    metrics_real_df = compute_metrics_with_moves(
        labors_hist_df, moves_hist_df, fechas, dist_dict,
        workday_hours=8, city=city, assignment_type='historic',
        skip_weekends=False, dist_method='haversine'
    )
    metrics_algo_df = compute_metrics_with_moves(
        labors_algo_offline_df, moves_algo_offline_df, fechas, dist_dict,
        workday_hours=8, city=city, assignment_type='algorithm',
        skip_weekends=False, dist_method='haversine'
    )

    if group_by is not None:
        metrics_real_df = add_aggregated_totals(metrics_real_df, group_by=group_by)
        metrics_algo_df = add_aggregated_totals(metrics_algo_df, group_by=group_by)

    # --- Labels ---
    labels = {
        "vt_count": "Labores tipo Vehicle Transportation",
        "num_drivers": "Número de conductores",
        "labores_por_conductor": "Labores por conductor",
        "utilizacion_promedio_%": "Utilización promedio (%)",
        "total_distance": "Distancia total (km)",
        "driver_move_distance": "Distancia recorrida por conductores (km)",
        "labor_extra_time": "Tiempo extra total por labor (min)",
        "driver_extra_time": "Tiempo extra total por conductor (min)",
    }

    if isinstance(metricas, str):
        metricas = [metricas]

    # --- Eje X ---
    x_real = pd.to_datetime(metrics_real_df["day"])
    x_alg = pd.to_datetime(metrics_algo_df["day"])

    if xaxis_mode == "day":
        # Crear etiquetas 'day1', 'day2', ...
        day_labels = [f"day{i+1}" for i in range(len(x_real))]
        x_real_display = day_labels
        x_alg_display = day_labels
    else:
        # Usar fechas reales
        x_real_display = x_real
        x_alg_display = x_alg

    # 🎨 Colors & styles
    color_real, color_alg = "#800080", "#17BECF"

    figs = []
    for i in range(0, len(metricas), 2):
        pair = metricas[i:i+2]
        fig = make_subplots(
            rows=1, cols=len(pair),
            subplot_titles=[labels.get(m, m) for m in pair]
        )

        for j, metrica in enumerate(pair, start=1):
            label = labels.get(metrica, metrica)

            # Serie real
            fig.add_trace(go.Scatter(
                x=x_real_display, y=metrics_real_df[metrica],
                mode="lines+markers",
                name=f"{label} (Real)",
                line=dict(color=color_real, dash="solid"),
                marker=dict(symbol="circle", size=8, color=color_real),
                legendgroup=f"{metrica}_real",
                showlegend=True
            ), row=1, col=j)

            # Serie algoritmo
            fig.add_trace(go.Scatter(
                x=x_alg_display, y=metrics_algo_df[metrica],
                mode="lines+markers",
                name=f"{label} (Algoritmo)",
                line=dict(color=color_alg, dash="dash"),
                marker=dict(symbol="x", size=8, color=color_alg),
                legendgroup=f"{metrica}_alg",
                showlegend=True
            ), row=1, col=j)

            fig.update_xaxes(title="Día", row=1, col=j)
            fig.update_yaxes(title=label, row=1, col=j)

        fig.update_layout(
            height=500,
            width=1000,
            title=f"Métricas en {get_city_name_from_code(city)} ({fechas[0]} a {fechas[1]})",
            margin=dict(t=100, b=100),
            legend=dict(
                orientation="h",
                yanchor="top", y=-0.25,
                xanchor="center", x=0.5
            )
        )

        fig.show()
        figs.append(fig)

    return figs

In [ ]:
metricas = ['vt_count', 'num_drivers', "labores_por_conductor", 'total_distance', 
            'labor_extra_time', 'driver_extra_time', 'driver_move_distance']
# metricas = ['driver_move_distance']


def plot_metrics_comparison_interactive(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_df: pd.DataFrame,
    moves_algo_df: pd.DataFrame,
    labors_algo_online_df: pd.DataFrame,
    moves_algo_online_df: pd.DataFrame,
    cities: list[str],
    metrics: list[str], 
    metricas,
    dist_dict: dict,
    fechas: tuple[str, str]
):
    """
    Interactive version of plot_metrics_comparison with ipywidgets.
    Allows selecting the city dynamically from a dropdown.
    """

    def _plot_for_city(city: str, 
                       metric,
                       ):

        alphas_selection = {'hybrid':0.1,
                            'driver_extra_time':0.1, 
                            'driver_distance':0}
        alpha = alphas_selection[metric]
        num_iter=500

        labors_algo_temp = labors_algo_df[(labors_algo_df['metric']==metric) &
                       (labors_algo_df['alpha']==alpha) &
                       (labors_algo_df['num_iter']==num_iter)]
        moves_algo_temp = moves_algo_df[(moves_algo_df['metric']==metric) &
                       (moves_algo_df['alpha']==alpha) &
                       (moves_algo_df['num_iter']==num_iter)]
        
        bbb = labors_algo_temp.copy()
        bbb['city'] = 'ALL'
        labors_algo_temp = pd.concat([labors_algo_temp, bbb])

        
        ddd = moves_algo_temp.copy()
        ddd['city'] = 'ALL'
        moves_algo_temp = pd.concat([moves_algo_temp, ddd])
        
        
        # Call your existing function
        figs = plot_metrics_comparison(
            labors_hist_df=labors_hist_df,
            moves_hist_df=moves_hist_df,
            labors_algo_df=labors_algo_temp,
            moves_algo_df=moves_algo_temp,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas, 
            # group_by=['alpha', 'num_iterations', 'metric']
        )
        # return figs

    # Create dropdown
    city_dropdown = widgets.Dropdown(
        options=['ALL'] + cities,
        value=cities[0],  # default city
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    metrics_dropdown = widgets.Dropdown(
        options=metrics,
        value=metrics[0],  # default city
        description="Métrica:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # Link dropdown with plotting function
    interact(_plot_for_city, city=city_dropdown, metric=metrics_dropdown)

plot_metrics_comparison_interactive(
    labors_hist_df,
    moves_hist_df,
    labors_algo_offline_df,
    moves_algo_offline_df,
    labors_algo_online_df,
    moves_algo_online_df,
    cities=valid_cities,  # put the city codes here
    metrics=metrics, 
    # alphas=alphas,
    # iterations_nums=iterations_nums,
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=(fechas[0], fechas[-1])
)

interactive(children=(Dropdown(description='Ciudad:', index=1, layout=Layout(width='300px'), options=('ALL', '…